# How to use Model

In [1]:
import sys
sys.path.append("/image_models/timm")

In [5]:
from models import efficientnet
import timm

## Load pretrained model

In [8]:
model = timm.create_model('mobilenetv3_large_100', pretrained=True)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mobilenetv3_large_100_ra-f55367f5.pth" to C:\Users\WorkStation_ASUS/.cache\torch\hub\checkpoints\mobilenetv3_large_100_ra-f55367f5.pth


## List Model with pretreind weight

In [10]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

['adv_inception_v3',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'cspdarknet53',
 'cspresnet50',
 'cspresnext50',
 'deit_base_distilled_patch16_224',
 'deit_base_distilled_patch16_384',
 'deit_base_patch16_224',
 'deit_base_patch16_384',
 'deit_small_distilled_patch16_224',
 'deit_small_patch16_224',
 'deit_tiny_distilled_patch16_224',
 'deit_tiny_patch16_224',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2next',
 'dla60x',
 'dla60x_c',
 'dla102',
 'dla102x',
 'dla102x2',
 'dla169',
 'dm_nfnet_f0',
 'dm_nfnet_f1',
 'dm_nfnet_f2',
 'dm_n

## List Model Architectures by Wildcard

In [11]:
import timm
from pprint import pprint
model_names = timm.list_models('*resne*t*')
pprint(model_names)

['bat_resnext26ts',
 'cspresnet50',
 'cspresnet50d',
 'cspresnet50w',
 'cspresnext50',
 'cspresnext50_iabn',
 'eca_lambda_resnext26ts',
 'ecaresnet26t',
 'ecaresnet50d',
 'ecaresnet50d_pruned',
 'ecaresnet50t',
 'ecaresnet101d',
 'ecaresnet101d_pruned',
 'ecaresnet200d',
 'ecaresnet269d',
 'ecaresnetlight',
 'ecaresnext26t_32x4d',
 'ecaresnext50t_32x4d',
 'ens_adv_inception_resnet_v2',
 'gcresnet50t',
 'gcresnext26ts',
 'geresnet50t',
 'gluon_resnet18_v1b',
 'gluon_resnet34_v1b',
 'gluon_resnet50_v1b',
 'gluon_resnet50_v1c',
 'gluon_resnet50_v1d',
 'gluon_resnet50_v1s',
 'gluon_resnet101_v1b',
 'gluon_resnet101_v1c',
 'gluon_resnet101_v1d',
 'gluon_resnet101_v1s',
 'gluon_resnet152_v1b',
 'gluon_resnet152_v1c',
 'gluon_resnet152_v1d',
 'gluon_resnet152_v1s',
 'gluon_resnext50_32x4d',
 'gluon_resnext101_32x4d',
 'gluon_resnext101_64x4d',
 'gluon_seresnext50_32x4d',
 'gluon_seresnext101_32x4d',
 'gluon_seresnext101_64x4d',
 'ig_resnext101_32x8d',
 'ig_resnext101_32x16d',
 'ig_resnext101_

# efficent net

## dataset

In [1]:
from util.dataloader import DataTransform 
from util.dataloader import CowDataset
from util.dataloader import makelabels
from util.preprocessing import read_data
import torch.utils.data as data

In [2]:
train_path="/Project/Data/DATA5/3D_TRAIN_DATA"
val_path="/Project/Data/DATA5/3D_VAL_DATA"

train_files,val_files=read_data(train_path,val_path)
classindex=makelabels(train_path)

In [3]:
batch_size=256

train_dataset=CowDataset(train_files, phase="train", transform=DataTransform(),classindex=classindex)
val_dataset=CowDataset(val_files, phase="val", transform=DataTransform(),classindex=classindex)

train_dataloader=data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# 辞書型変数
dataloaders_dict = {"train": train_dataloader, "val":val_dataloader}

## net

In [4]:
from torch import nn
import random
import math
import time
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim

import timm
import sys
sys.path.append("/image_models/timm")
from models import efficientnet

In [5]:
# 初期設定
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [6]:
net = timm.create_model('efficientnet_b0',pretrained=True)
net.classifier=nn.Linear(1280,49)

In [7]:
optimizer = optim.SGD(net.parameters(),lr=1e-3)#,weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

## training

In [8]:
from tensorboardX import SummaryWriter
writer = SummaryWriter("./tbX/")

In [9]:
from sklearn.metrics import confusion_matrix

def train_model(net,dataloaders_dict,criterion,optimizar,num_epochs):

    #check GPU
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Device:",device)

    #netをデバイスへ移動
    net.to(device)
    #net2.to(device)

    conf_mem=[]
    #ネットがある程度固定であれば,高速化
    torch.backends.cudnn.benchmark = True

    #epoch roop
    for epoch in range(num_epochs):
        #start time
        t_epoch_start=time.time()

        print("----------")
        print("Epoch{}/{}".format(epoch+1,num_epochs))
        print("----------")

        #train val roop
        for phase in ["train","val"]:
            if phase == "train":
                net.train()
                optimizar.zero_grad()

            else:
                net.eval()

            # 損失和
            epoch_loss = 0.0
            # 正解数
            epoch_corrects = 0
            #Top5
            correct_k = 0

            predlist=torch.zeros(0,dtype=torch.long, device='cpu')
            lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

            for images,labels in dataloaders_dict[phase]:
                #Note! batchsize 1 occur error,
                if images.size()[0] == 1:
                    continue

                #initialize optimizer
                optimizer.zero_grad()

                #to device
                images=images.to(device)
                labels = labels.to(device)

                #forward propagation
                with torch.set_grad_enabled(phase=="train"):
                    outputs=net(images)
                    loss=criterion(outputs,labels)
                    _,preds=torch.max(outputs,1)

                    #back propagation
                    if phase=="train":
                        loss.backward()
                        optimizer.step()
                    else:
                        predlist=torch.cat([predlist,preds.view(-1).cpu()])
                        lbllist=torch.cat([lbllist,labels.view(-1).cpu()])


                    # lossの合計を更新
                    epoch_loss += loss.item() * images.size(0)
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

                    #Top5の正解数を更新
                    _, indices = outputs.topk(5, dim = 1,  largest=True, sorted=True)
                    prediction=indices.t()
                    top5_correct = prediction.eq(labels.view(1, -1).expand_as(prediction))
                    correct_k += top5_correct[:5].reshape(-1).float().sum(0, keepdim=True)

                    #calculate
                    epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
                    epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
                    epoch_top5_acc = correct_k.double() / len(dataloaders_dict[phase].dataset)
                    epoch_top5_acc = epoch_top5_acc.to('cpu').detach().numpy().copy()

            print('{} Loss: {:.4f} Top1-Acc: {:.4f} Top5-Acc: {:.4f}' .format(phase, epoch_loss, epoch_acc,epoch_top5_acc[0]))

            if phase == "train":
                writer.add_scalar("TRAIN_Loss",epoch_loss,epoch)
                writer.add_scalar("TRAIN_acc",epoch_acc,epoch)
            else:
                #Confusion matrix
                conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
                conf_mem.append(conf_mat)

                writer.add_scalar("VAL_Loss",epoch_loss,epoch)
                writer.add_scalar("VAL_acc",epoch_acc,epoch)


        t_epoch_finish=time.time()
        print("Timer: {:.4f} sec.".format(t_epoch_finish - t_epoch_start))
        t_epoch_start=time.time()
    return conf_mem

In [10]:
# 学習・検証を実行
num_epochs = 100
conf_mem=train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

Device: cuda:0
----------
Epoch1/100
----------
train Loss: 0.0196 Top1-Acc: 0.0249 Top5-Acc: 0.1105
val Loss: 0.1986 Top1-Acc: 0.0102 Top5-Acc: 0.1019
Timer: 77.8183 sec.
----------
Epoch2/100
----------
train Loss: 0.0194 Top1-Acc: 0.0293 Top5-Acc: 0.1241
val Loss: 0.1993 Top1-Acc: 0.0190 Top5-Acc: 0.0876
Timer: 73.9217 sec.
----------
Epoch3/100
----------
train Loss: 0.0195 Top1-Acc: 0.0277 Top5-Acc: 0.1285
val Loss: 0.1982 Top1-Acc: 0.0199 Top5-Acc: 0.0996
Timer: 71.1622 sec.
----------
Epoch4/100
----------
train Loss: 0.0193 Top1-Acc: 0.0351 Top5-Acc: 0.1391
val Loss: 0.1988 Top1-Acc: 0.0204 Top5-Acc: 0.1135
Timer: 70.0862 sec.
----------
Epoch5/100
----------
train Loss: 0.0193 Top1-Acc: 0.0369 Top5-Acc: 0.1556
val Loss: 0.2001 Top1-Acc: 0.0232 Top5-Acc: 0.1140
Timer: 73.6335 sec.
----------
Epoch6/100
----------
train Loss: 0.0192 Top1-Acc: 0.0375 Top5-Acc: 0.1635
val Loss: 0.1989 Top1-Acc: 0.0232 Top5-Acc: 0.1228
Timer: 74.6173 sec.
----------
Epoch7/100
----------
train Loss